In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!brew upgrade
!brew install openjdk@11 
!pip install py4j
!pip install findspark
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install pyspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/local/Cellar/openjdk@11/11.0.12"
os.environ["SPARK_HOME"] = "/Users/davidgae/project16/spark-3.0.3-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

To reinstall 11.0.12, run:
  brew reinstall openjdk@11


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-13 06:48:24--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 2001:4800:3e1:1::228, 72.32.157.228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|2001:4800:3e1:1::228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  1.62MB/s    in 0.6s    

2021-12-13 06:48:25 (1.62 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.head(1)

[Row(marketplace='US', customer_id=22480053, review_id='R28HBXXO1UEVJT', product_id='0843952016', product_parent=34858117, product_title='The Rising', product_category='Books', star_rating=5, helpful_votes=0, total_votes=0, vine='N', verified_purchase='N', review_headline='Great Twist on Zombie Mythos', review_body="I've known about this one for a long time, but just finally got around to reading it for the first time.  I enjoyed it a lot!  What I liked the most was how it took a tired premise and breathed new life into it by creating an entirely new twist on the zombie mythos.  A definite must read!", review_date='2012-05-03')]

In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(4)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   47098686|             1|
|    9973109|             1|
|   52477231|            24|
|   27568383|             1|
+-----------+--------------+
only showing top 4 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1419701630|Chuck Close: Face...|
|0451216954|Dark Lover (Black...|
|0552142395|My Feudal Lord: A...|
|1931514941|   Love Hina, Vol. 1|
|0615462219|The Great Pain De...|
+----------+--------------------+
only showing top 5 rows



In [56]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R28HBXXO1UEVJT|   22480053|0843952016|      34858117| 2012-05-03|
| RZKRFS2UUMFFU|   44244451|031088926X|     676347131| 2012-05-03|
|R2WAU9MD9K6JQA|   20357422|0615268102|     763837025| 2012-05-03|
|R36SCTKYTVPZPC|   13235208|1900869225|     785539232| 2012-05-03|
|R10BM6JUOJX27Q|   26301786|1565129938|      64646125| 2012-05-03|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [115]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes","vine","verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28HBXXO1UEVJT|          5|            0|          0|   N|                N|
| RZKRFS2UUMFFU|          5|           15|         15|   N|                Y|
|R2WAU9MD9K6JQA|          3|            6|          8|   N|                N|
|R36SCTKYTVPZPC|          5|           10|         11|   N|                Y|
|R10BM6JUOJX27Q|          3|            0|          0|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [143]:
## Configure settings for RDS
#        "mode = \"append\"\n",
#        "jdbc_url=\"jdbc:postgresql://<endpoint>:5432/<database name>\"\n",
#        "config = {\"user\":\"postgres\", \n",
#        "          \"password\": \"<password>\", \n",
#        "          \"driver\":\"org.postgresql.Driver\"}"
#      ],

server = "database-2.ci4arflqa4bf.us-west-1.rds.amazonaws.com"
#server="database-3.ci4arflqa4bf.us-west-1.rds.amazonaws.com"
database = "my_first_db"
port = "5432"
username = "postgres"
password = "password134"

conn = f"postgres://{username}:{password}@{server}:{port}/{database}"

from sqlalchemy import create_engine 
engine = create_engine(conn, echo=False)

In [145]:
# Write review_id_df to table in RDS
pd_review_id_df=review_id_df.toPandas()
pd_review_id_df.head
pd_review_id_df.to_sql("review_id_table", con=engine)

#review_id_df.write.jdbc(jdbc_url, table='review_id_table', properties=config)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:54018)
Traceback (most recent call last):
  File "/Users/davidgae/project16/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/davidgae/project16/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:54018)

In [146]:
# Write products_df to table in RDS
# about 3 min
pd_products_df = products_df.toPandas()
pd_products_df.set_index("product_id ", inplace=True)
pd_products_df.head()
pd_products_df.to_sql("products", con=engine)
#products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:54018)
Traceback (most recent call last):
  File "/Users/davidgae/project16/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/davidgae/project16/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:54018)

In [147]:
# Write customers_df to table in RDS
# 5 min 14 s
pd_customers_df = customers_df.toPandas()
pd_customers_df.set_index("customer_id", inplace=True)
pd_customers_df.head()
pd_customers_df.to_sql("customers", con=engine)
#customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:54018)
Traceback (most recent call last):
  File "/Users/davidgae/project16/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/davidgae/project16/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:54018)

In [148]:
# Write vine_df to table in RDS
# 11 minutes
pd_vine_df = vine_df.toPandas()
pd_vine_df.set_index("review_id", inplace=True)
pd_vine_df.head()
pd_vine_df.to_sql("vine_table", con=engine)
#vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:54018)
Traceback (most recent call last):
  File "/Users/davidgae/project16/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/davidgae/project16/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:54018)